In [3]:
import copy
from scipy import stats
import numpy as np
import sampler.sampling as sampling
from nas_201_api import NASBench201API as api

nb201_api = api('./NAS-Bench-201-v1_0-e61699.pth')
print('done.')

try to create the NAS-Bench-201 api from ./NAS-Bench-201-v1_0-e61699.pth
done.


In [4]:

def get_acc_valid(dataset, index):
    """dataset in 'cifar10', 'cifar100', 'imgagenet'."""
    strings = nb201_api.query_by_arch(nb201_api.arch(index))
    strings = strings.split('\n')
    if dataset == 'cifar10':
        cifar10_valid_res = strings[3]
        startpoint = cifar10_valid_res.find('valid : [loss = ') + len('valid : [loss = ')
        toppoint = cifar10_valid_res.find('top1 = ', startpoint) + len('top1 = ')
        endpoint = cifar10_valid_res.find('%]', toppoint)
        ans = cifar10_valid_res[toppoint:endpoint]
        return float(ans)
    elif dataset == 'cifar100':
        cifar100_res = strings[7]
        startpoint = cifar100_res.find('valid : [loss = ') + len('valid : [loss = ')
        toppoint = cifar100_res.find('top1 = ', startpoint) + len('top1 = ')
        endpoint = cifar100_res.find('%]', toppoint)
        ans = cifar100_res[toppoint:endpoint]
        return float(ans)
    elif dataset == 'imagenet':
        imagenet_res = strings[9]
        startpoint = imagenet_res.find('valid : [loss = ') + len('valid : [loss = ')
        toppoint = imagenet_res.find('top1 = ', startpoint) + len('top1 = ')
        endpoint = imagenet_res.find('%]', toppoint)
        ans = imagenet_res[toppoint:endpoint]
        return float(ans)
    else:
        print('dataset error')
        exit(1)

def get_acc_test(dataset, index):
    """dataset in 'cifar10', 'cifar100', 'imgagenet'."""
    strings = nb201_api.query_by_arch(nb201_api.arch(index))
    strings = strings.split('\n')
    if dataset == 'cifar10':
        cifar10_test_res = strings[5]
        startpoint = cifar10_test_res.find('test  : [loss = ') + len('test  : [loss = ')
        toppoint = cifar10_test_res.find('top1 = ', startpoint) + len('top1 = ')
        endpoint = cifar10_test_res.find('%]', toppoint)
        ans = cifar10_test_res[toppoint:endpoint]
        return float(ans)
    elif dataset == 'cifar100':
        cifar100_res = strings[7]
        startpoint = cifar100_res.find('test : [loss = ') + len('test : [loss = ')
        toppoint = cifar100_res.find('top1 = ', startpoint) + len('top1 = ')
        endpoint = cifar100_res.find('%]', toppoint)
        ans = cifar100_res[toppoint:endpoint]
        return float(ans)
    elif dataset == 'imagenet':
        imagenet_res = strings[9]
        startpoint = imagenet_res.find('test : [loss = ') + len('test : [loss = ')
        toppoint = imagenet_res.find('top1 = ', startpoint) + len('top1 = ')
        endpoint = imagenet_res.find('%]', toppoint)
        ans = imagenet_res[toppoint:endpoint]
        return float(ans)
    else:
        print('dataset error')
        exit(1)


In [5]:
cifar10_valid = []
cifar100_valid = []
imagenet_valid = []

cifar10_test = []
cifar100_test = []
imagenet_test = []

for i in range(15625):
    cifar10_valid.append(get_acc_valid('cifar10', i))
    cifar100_valid.append(get_acc_valid('cifar100', i))
    imagenet_valid.append(get_acc_valid('imagenet', i))
    cifar10_test.append(get_acc_test('cifar10', i))
    cifar100_test.append(get_acc_test('cifar100', i))
    imagenet_test.append(get_acc_test('imagenet', i))
    

cifar10_valid_sort = copy.deepcopy(cifar10_valid)
cifar10_valid_sort.sort(reverse=True)
cifar100_valid_sort = copy.deepcopy(cifar100_valid)
cifar100_valid_sort.sort(reverse=True)
imagenet_valid_sort = copy.deepcopy(imagenet_valid)
imagenet_valid_sort.sort(reverse=True)

cifar10_test_sort = copy.deepcopy(cifar10_test)
cifar10_test_sort.sort(reverse=True)
cifar100_test_sort = copy.deepcopy(cifar100_test)
cifar100_test_sort.sort(reverse=True)
imagenet_test_sort = copy.deepcopy(imagenet_test)
imagenet_test_sort.sort(reverse=True)

In [5]:
res = [2,3,3,1,0,3]


import numpy as np

def array2genostr(arr):
    OPS = ["none", "skip_connect", "nor_conv_1x1", "nor_conv_3x3", "avg_pool_3x3"]
    idx = [list(i).index(1.) for i in arr]
    op = [OPS[x] for x in idx]
    mixed = '|' + op[0] + '~0|+|' + op[1] + '~0|' + op[2] + '~1|+|' + op[3] + '~0|' + op[4] + '~1|' + op[5] + '~2|'
    return mixed

_tmp_np = np.array(res)
_tmp_oh = np.zeros((_tmp_np.size, 5))
_tmp_oh[np.arange(_tmp_np.size),_tmp_np] = 1
# print(_tmp_oh)
geno_str = array2genostr(_tmp_oh)
#     print(geno_str)
index = nb201_api.query_index_by_arch(geno_str)
# print(index)

print('valid')
print('acc_cifar10: {}, rank_cifar10: {}'.format(cifar10_valid[index], cifar10_valid_sort.index(cifar10_valid[index])))
print('acc_cifar100: {}, rank_cifar100: {}'.format(cifar100_valid[index], cifar100_valid_sort.index(cifar100_valid[index])))
print('acc_imgnet16: {}, rank_imgnet16: {}'.format(imagenet_valid[index], imagenet_valid_sort.index(imagenet_valid[index])))

print('test')
print('acc_cifar10: {}, rank_cifar10: {}'.format(cifar10_test[index], cifar10_test_sort.index(cifar10_test[index])))
print('acc_cifar100: {}, rank_cifar100: {}'.format(cifar100_test[index], cifar100_test_sort.index(cifar100_test[index])))
print('acc_imgnet16: {}, rank_imgnet16: {}'.format(imagenet_test[index], imagenet_test_sort.index(imagenet_test[index])))


valid
acc_cifar10: 91.22, rank_cifar10: 43
acc_cifar100: 71.97, rank_cifar100: 45
acc_imgnet16: 45.59, rank_imgnet16: 71
test
acc_cifar10: 93.98, rank_cifar10: 48
acc_cifar100: 71.69, rank_cifar100: 79
acc_imgnet16: 45.82, rank_imgnet16: 66
